In [2]:
import nltk

In [3]:
nltk.download('vader_lexicon')
# Vadar = Valence Aware dictionary

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\BibhuJha\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [5]:
senti = SentimentIntensityAnalyzer()

In [6]:
senti.polarity_scores('This is a good movie')

{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}

In [7]:
senti.polarity_scores('This was the best,most awesome movie EVER MADE!!! ')

{'neg': 0.0, 'neu': 0.585, 'pos': 0.415, 'compound': 0.7163}

In [8]:
import pandas as pd

In [10]:
ds = pd.read_csv('hotel_review.tsv',sep='\t')

In [11]:
ds.isnull().sum()

Description    0
Is_Response    0
dtype: int64

In [12]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38932 entries, 0 to 38931
Data columns (total 2 columns):
Description    38932 non-null object
Is_Response    38932 non-null object
dtypes: object(2)
memory usage: 608.4+ KB


In [19]:
ds.columns = ['Message','Verdict']
ds.head()

ValueError: Length mismatch: Expected axis has 3 elements, new values have 2 elements

In [18]:
ds['Sentiment_scores'] = ds['Message'].apply(lambda msg: senti.polarity_scores(msg))

In [28]:
ds['%Postive'] = ds.Sentiment_scores.apply(lambda senti: senti['pos'])
ds['%Negative'] = ds.Sentiment_scores.apply(lambda senti: senti['neg'])
ds['%Neutral'] = ds.Sentiment_scores.apply(lambda senti: senti['neu'])
ds['Sentiment'] = ds.Sentiment_scores.apply(lambda senti: 'happy' if senti['compound']>0 else 'not happy')
ds.drop('Sentiment_scores',inplace=True,axis=1)
ds.head()


,Message,Verdict,%Postive,%Negative,%Neutral,Sentiment
0,The room was kind of clean but had a VERY stro...,not happy,0.325,0.000,0.675,happy
1,I stayed at the Crown Plaza April -- - April -...,not happy,0.050,0.053,0.897,not happy
2,I booked this hotel through Hotwire at the low...,not happy,0.085,0.048,0.866,happy
3,Stayed here with husband and sons on the way t...,happy,0.347,0.000,0.653,happy
4,My girlfriends and I stayed here to celebrate ...,not happy,0.088,0.041,0.871,happy


In [29]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [30]:
confusion_matrix(ds.Verdict, ds.Sentiment)

array([[26084,   437],
       [ 8585,  3826]], dtype=int64)

In [32]:
print(classification_report(ds.Verdict, ds.Sentiment))

              precision    recall  f1-score   support

       happy       0.75      0.98      0.85     26521
   not happy       0.90      0.31      0.46     12411

    accuracy                           0.77     38932
   macro avg       0.82      0.65      0.66     38932
weighted avg       0.80      0.77      0.73     38932



In [33]:
accuracy_score(ds.Verdict, ds.Sentiment)

0.7682626117332786

In [42]:
ds.head()

,Message,Verdict,%Postive,%Negative,%Neutral,Sentiment
0,The room was kind of clean but had a VERY stro...,not happy,0.325,0.000,0.675,happy
1,I stayed at the Crown Plaza April -- - April -...,not happy,0.050,0.053,0.897,not happy
2,I booked this hotel through Hotwire at the low...,not happy,0.085,0.048,0.866,happy
3,Stayed here with husband and sons on the way t...,happy,0.347,0.000,0.653,happy
4,My girlfriends and I stayed here to celebrate ...,not happy,0.088,0.041,0.871,happy


In [52]:
# Applying Tfidf text Classification method
X = ds['Message'].values
y = ds['Verdict'].values

In [53]:
from sklearn.model_selection import train_test_split

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [55]:
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression

In [56]:
pipe = Pipeline([('Tfidf',TfidfVectorizer()),('clif',LogisticRegression())])

In [57]:
pipe.fit(X_train,y_train)

C:\Users\BibhuJha\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('Tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clif',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scalin

In [65]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [62]:
confusion_matrix(y_test,pipe.predict(X_test))

array([[8226,  535],
       [ 954, 3133]], dtype=int64)

In [64]:
print(classification_report(y_test,pipe.predict(X_test)))

              precision    recall  f1-score   support

       happy       0.90      0.94      0.92      8761
   not happy       0.85      0.77      0.81      4087

    accuracy                           0.88     12848
   macro avg       0.88      0.85      0.86     12848
weighted avg       0.88      0.88      0.88     12848



In [67]:
accuracy_score(y_test,pipe.predict(X_test))

0.8841064757160647

In [72]:
ds['Sentiment_evaluation_using_Tfidf'] = pipe.predict(X)

In [73]:
ds.head()

,Message,Verdict,%Postive,%Negative,%Neutral,Sentiment,Sentiment_evaluation_using_Tfidf
0,The room was kind of clean but had a VERY stro...,not happy,0.325,0.000,0.675,happy,not happy
1,I stayed at the Crown Plaza April -- - April -...,not happy,0.050,0.053,0.897,not happy,not happy
2,I booked this hotel through Hotwire at the low...,not happy,0.085,0.048,0.866,happy,not happy
3,Stayed here with husband and sons on the way t...,happy,0.347,0.000,0.653,happy,happy
4,My girlfriends and I stayed here to celebrate ...,not happy,0.088,0.041,0.871,happy,not happy
